# "Shazam"

Merci de remplir les informations ci-dessous, pour attribuer les notes :

In [ ]:
# N° de binôme (ex. binome_e99)
# Nom des étudiants

## Consignes importantes

Vous aurez des manipulations à faire dans BigQuery, des requêtes à mettre au point, et des réponses à trouver. Je note les requêtes et les réponses. Pour les requêtes, vous avez 2 possibilités :
- soit les mettre au point directement dans le notebook
- soit les mettre au point dans l'interface de BigQuery, auquel cas **vous devrez les recopier dans le notebook**. BigQuery tout seul ne sauve pas les requêtes et je ne regarde pas ce qu'il y a dans BigQuery. On doit pouvoir réexécuter le notebook d'un coup sans erreur.

Il y a un notebook "Prise en main" qui n'est pas à rendre, mais vous aidera de temps en temps.

Déroulé conseillé :
- suivre la partie I du notebook "Prise en main"
- faire la partie 1 de ce notebook
- suivre la partie II de Prise en main"
- puis faire les parties 2 et 3 de ce notebook, en vous référant à la partie III de "Prise en main" lorsque nécessaire

Dépendance entre les parties de ce notebook :
- la partie 1 est nécessaire pour la suite
- les parties 1 et 2 jusqu'à la question "Enrichissement de la base de données" sont nécessaires pour la partie 3
- sauf la toute dernière question de la partie 3, qui est indépendante

Si vous êtes bloqués : le dataset `shared` contient les tables que vous devez créer avec les requêtes, avec le préfixe `corrige__` (deux "tirets du 8"). Vous pouvez donc sauter une question en changeant le nom de la table (ex. `SELECT * FROM binome_xxx.raw_vectors` -> `SELECT * FROM shared.corrige__raw_vectors`). Le corrigé de l'UDF n'est pas donné en revanche.

#### Merci de ne faire qu'un notebook "TP3" par binôme.

#### A la fin du TP, avant de partir, sauvez ce notebook sur l'ordinateur, et envoyez-moi le fichier `TP3.ipynb` à l'adresse tvial@octo.com.

## Contexte et données

Vous développez un service de reconnaissance musicale (type Shazam), basé sur l’utilisation d’**embeddings** (vecteurs) censés représenter les “empreintes digitales” des morceaux. La recherche de morceaux proches d'un autre revient ainsi à calculer les distances entre des embeddings.

Les données sont constituées de morceaux composés par des artistes ; chaque morceau a préalablement été transformé en un certain nombre de vecteurs, dont le nombre peut varier. En effet un vecteur encode environ 10 s de musique, un morceau de 200 secondes aura ainsi une vingtaine de vecteurs, et un morceau de 400 secondes une quarantaine.

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/vectors.png)

### Informations sur les morceaux

Le premier jeu de données est un fichier `metadata.csv`, qui énumère les un peu plus de 1400 morceaux. Il contient une ligne par morceau avec les champs suivants séparés par des `“;”` :
- `song_id` : identifiant numérique du morceau
- `artist` : nom de l’artiste
- `title` : titre du morceau
- `duration` : durée du morceau, en secondes (NB : cette information est fictive)

### Vecteurs d'embedding

Le deuxième jeu de données est un ensemble de fichiers “JSONL”, c’est-à-dire des groupes de documents JSON stockés dans un même fichier avec un document par ligne. Ces fichiers correspondent aux vecteurs issus des morceaux. Il y a au total 1367 documents, regroupés en 18 fichiers `vectors_001.jsonl` à `vectors_018.jsonl`.

Il y a moins de documents que de morceaux (1367 contre 1418), car le calcul des embeddings a échoué pour quelques uns d'entre eux. Ce n'est pas gênant pour le TP.

Chaque document JSON (une ligne de fichier JSONL donc) est structuré ainsi :
- Un champ `“id”` qui contient l’ID du morceau (le même que dans le fichier `metadata.csv`)
- Un champ `“vectors”` qui est un tableau d’objets :
  - Chacun de ces objets a un seul champ `“vector”`, qui est un tableau de 16 nombres flottants
  - Tous les vecteurs sont donc de la même taille (16), mais il y a un nombre variable de tels vecteurs pour un morceau donné

En pratique, voici à quoi ressemble un fichier JSONL :
```
{“id”: 12, “vectors”: [{“vector”: [0.1, 0.2, ...]}, {“vector”: [0.4, 0.6, ...]}, ...]}
{“id”: 13, “vectors”: [{“vector”: [0.6, 0.5, ...]}, {“vector”: [0.7, 0.9, ...]}, ...]}
...
```

Les vecteurs ont été produits ainsi :
- lecture de fichiers MP3
- tranformation en embeddings avec le modèle [YAMNet](https://www.kaggle.com/models/google/yamnet)
- réduction du volume :
  - agrégation des fenêtres (10 vecteurs pour 1 seconde de musique => 1 vecteur pour 10 secondes, moyenne des 10 vecteurs)
  - ACP pour diminuer la taille des vecteurs de 1024 à 16

### Vecteurs de requêtes

Enfin, il y a 3 morceaux inconnus qui seront confrontés à la base de données des vecteurs : ce seront les "requêtes" de Shazam. Elles sont dans un seul fichier JSONL, avec la même structure que les précédents (le champ `"id"` représente l'ID de requête et non plus de morceau).

Toutes les données d’entrée sont dans le système de stockage objet de GCP, Google Cloud Storage, et nous allons les importer dans Big Query.

# Partie 1 : Manipulation dans BigQuery

Dans cette partie, on ne fait que des manipulations dans l'interface de BigQuery. Le notebook sert juste de guide et je n'attends pas de requête.

Prenez soin de respecter les étapes, car les données chargées serviront de base aux requêtes.

## Import des informations sur les morceaux

A l’aide de l’interface graphique, créer une table `metadata` dans votre dataset, dont les données se trouvent dans Google Cloud Storage.

- Après avoir choisi le type de source, un champ apparaît avec un bouton “PARCOURIR”. En cliquant dessus, il faut sélectionner le “bucket” (espace de stockage) `but-tp-shazam-datalake`, et à l’intérieur, le fichier `metadata.csv`.
- Le format doit être CSV
- Vous pouvez cocher “Détection automatique” dans la section “Schéma” pour que BigQuery trouve tout seul les champs et leurs types
- Attention, le séparateur de champs est un point-virgule dans le fichier, mais BigQuery suppose une virgule par défaut. Trouvez où changer ce paramètre
- Ne changez pas les autres paramètres par défaut
- Cliquez enfin sur “CREER LA TABLE” pour lancer le “job” de chargement

Si tout s’est bien passé, vous ne devez pas avoir d’erreurs. Sinon, vérifiez bien les informations.

En cliquant sur le nom de la table dans l’explorateur, vous pouvez voir la structure que Big Query a déduite, et en cliquant sur “PREVIEW”, avoir un extrait des données. Vérifiez bien que le contenu semble correct, sinon il faut supprimer la table (“Supprimer” depuis les 3 petits points de l’explorateur) et recommencer le chargement.

## Import des vecteurs d'embedding

Créer de même une nouvelle table, `raw_vectors`, qui va cette fois contenir les vecteurs bruts tirés des documents JSON.

- Ces données sont au format JSONL
- BigQuery est capable de détecter le schéma comme pour metadata
- Elles se trouvent dans le même bucket que `metadata.csv`, mais dans le répertoire `tracks`. Vous pouvez sélectionner un des fichiers du bucket, et remplacer ensuite son nom par un nom générique (ex. `vectors_001.jsonl` → `*.jsonl`) pour que BigQuery importe tout d’un coup

Comme précédemment, vérifiez le résultat via l’explorateur. La structure doit ressembler à ceci :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/struct_raw_vectors.png)

Et la prévisualisation n’est pas très lisible, à cause des tableaux imbriqués dans des objets :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/preview_raw_vectors.png)

## Import des requêtes

Procéder de même pour les requêtes, dans le répertoire `queries`, pour créer une table `raw_queries`.

# Partie 2 : Requêtes dans le notebook

Dans cette partie, il faut écrire les requêtes dans le notebook.

## Configuration de l'extension BigQuery.

In [ ]:
%load_ext bigquery_magics

In [ ]:
import bigquery_magics
bigquery_magics.context.project = 'but-tp'

## Transposition des vecteurs

Actuellement, les données de vecteurs dans la table ont un modèle calqué sur celui des fichiers, qui n’est donc pas très pratique. On cherche à les “dénormaliser”, c’est-à-dire à transformer chaque ligne (qui correspond à un morceau) en autant de lignes qu’il y a de vecteurs pour son morceau, en répétant l’ID du morceau.

Schématiquement, cela revient à faire une opération de ce style pour chaque morceau (ici celui d’ID 42) :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/transposition.png)

Les ID sont répétés autant de fois qu’il y a de fenêtres et donc de vecteurs. Le nouveau champ `window_id` est le numéro de la fenêtre dans le morceau, i.e. la position du vecteur dans le tableau (le premier ayant la position 0).

Mettre au point une requête `SELECT` qui fait cette transposition, puis créer une table `flat_vectors` avec le résultat. La table doit avoir la structure suivante :
- `id` : l’ID du morceau
- `window_id` : numéro de la fenêtre
- `vector` : un vecteur unique = un tableau à 16 flottants

Vous pouvez vous aider du notebook de "Prise en main", dans la partie III. Il faut en plus accéder au sous-tableau `vector` qui se situe à l’intérieur des éléments du tableau principal (analogue à `myarray` dans l’exemple). L’équivalent de `element` est alors un objet JSON, dont on peut extraire l’attribut vector grâce à la syntaxe suivante : `element.vector` (à utiliser dans la liste des colonnes après le `SELECT`).

In [ ]:
%%bigquery
CREATE TABLE ... AS ...

Vous devez obtenir 70 124 lignes, et l’aperçu dans BigQuery doit ressembler à ceci (l’ordre des morceaux et fenêtres est arbitraire) :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/preview_flat_vectors.png)

## Transposition des vecteurs de `raw_queries`

Appliquer la même méthode pour créer une table `flat_queries`, à partir de `raw_queries`. La structure doit être la même.

In [ ]:
%%bigquery
...

La table doit contenir 256 éléments.

## Enrichissement de la base de données

Maintenant que nous avons des vecteurs à plat, nous allons enrichir la table avec les métadonnées des morceaux.

Créer une table `vectors_with_metadata`, avec comme structure :
- `song_id` (provient des deux tables)
- `window_id` (provient de `flat_vectors`)
- `artist` (provient de `metadata`)
- `title` (provient de `metadata`)
- `duration` (provient de `metadata`)
- `vector` (provient de `flat_vectors`)

In [ ]:
%%bigquery
...

**Question** : à combien de lignes peut-on s'attendre comme résultat, et pourquoi ?

In [ ]:
# Réponse

## Création d'une UDF

Pour les besoins de notre clone de Shazam, il faut être en mesure de calculer la distance euclidienne entre deux vecteurs de taille identique. Nous allons pour cela créer une UDF : _User-Defined Function_.

Dans le notebook de prise en main, vous avez un exemple d’UDF qui met en rapport les éléments de 2 vecteurs passés en paramètre, comme une sorte de jointure. Elle est aussi disponible dans vos environnements sous le nom `shared.join_vectors`.

Pour cette question, il est demandé de créer une autre UDF, `euclidean2`, qui retourne un flottant et non une table virtuelle, en l’occurrence le carré de la distance euclidienne entre 2 vecteurs qui lui sont passés en paramètre. Vous pourrez utiliser directement la fonction `join_vectors`, comme si son résultat était une table SQL avec 2 colonnes, une pour chaque vecteur, et une ligne par paire d’éléments. Ou vous pourrez vous en inspirer pour écrire `euclidean2` de zéro, comme vous le souhaitez.

Pour rappel, si $(x_i)$ et $(y_i)$ sont deux vecteurs, le carré de cette distance est $\sum_{i}{(x_i-y_i)^{2}}$. On ne cherche pas à appliquer une racine carrée pour avoir la distance absolue, le carré suffit.

%%bigquery
CREATE OR REPLACE FUNCTION binome_xxx.euclidean2 ...

Test de la fonction :

In [ ]:
%%bigquery
SELECT binome_xxx.euclidean2([3., 1., 2.], [4., 5., 6.])

Le résultat doit être 33.

## Calcul des distances par fenêtre

Ecrire une requête qui calcule, pour chaque morceau de `vectors_with_metadata`, le carré de la distance entre tous ses vecteurs et tous ceux de la table `flat_queries` (produit cartésien). Le résultat doit avoir les colonnes suivantes :
- `song_id_ref` (ID du morceau de référence provenant de `vectors_with_metadata`)
- `window_id_ref` (ID de la fenêtre du vecteur du morceau de référence)
- `query_id` (ID de la requête provenant de `flat_queries`)
- `window_id_query` (ID de la fenêtre du vecteur issu de `flat_queries`)
- `artist_ref`, `title_ref`, `duration_ref` : informations du morceau de référence
- `distance2` (carré de la distance)

Dans ce qui précède, `_ref` désigne donc chaque morceau de la base de référence (attention de ne pas mélanger).

Il n’est pas demandé de créer une table, en revanche il faut bien copier la requête dans la cellule ci-dessous.

Gardez la clause `LIMIT 100`, elle permet de n'envoyer que quelques lignes (100) au notebook ; on n'est pas intéressé par le détail pour l'instant.

**Attention, c'est bien un produit cartésien que l'on veut, il n'y a pas d'égalité entre les ID de morceaux et de queries, ils n'ont rien à voir !**

In [ ]:
%%bigquery
...
LIMIT 100

Pour contrôler, voici un les premières lignes du résultat pour `song_id_ref = 43` et  `query_id = 2`, triées par `window_id_ref` et `window_id_query` :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/check_query1.png)

**Question** : comment peut-on prédire le nombre de lignes du résultat ?

In [ ]:
# Réponse

## Rapprochement des fenêtres

Réutiliser la requête de la question précédente **sans la clause `LIMIT`** dans une clause `WITH` (voir notebook "Prise en main"), pour produire une requête qui calcule, pour chaque fenêtre de `queries` et chaque morceau de référence, la distance à la fenêtre la plus proche du morceau de référence.

Structure attendue du résultat :
- `query_id` (ID de la requête)
- `song_id_ref` (ID du morceau de référence comparé)
- `artist_ref`, `title_ref`, `duration_ref` : informations du morceau de référence
- `window_id_query` (ID de la fenêtre du vecteur requêté)
- `min_distance2` (carré de la distance la plus faible parmi les fenêtres du morceau de référence, pour un ensemble `query_id` + `window_id_query` + morceau de référence donné)

Rappel : il faut retirer la clause `LIMIT` de la requête réutilisée, mais on en ajoute une sur le résultat global.

In [ ]:
%%bigquery
...
LIMIT 100

## Détermination des morceaux les plus proches

Une fois ceci mis au point, proposer les morceaux les plus probables pour les 3 requêtes de `flat_queries`.

Pour ce faire, on peut :
- encapsuler la requête précédente dans une nouvelle clause `WITH`
- ... et l’utiliser dans une agrégation en calculant, pour chaque requête et chaque morceau de référence, la somme des `min_distance2`. Un tri sur le couple (somme des `min_distance2`, `query_id`) devrait vous remonter les morceaux candidats en premier.

Ainsi, la "distance" entre 2 morceaux est la somme des distances minimales entre les fenêtres des morceaux. Principe illustré ci-dessus, pour un seul couple (`query_id`, `song_id_ref`) :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/min_dist.png)

**Vérifiez que votre requête ne contient plus aucune clause `LIMIT`**.

In [ ]:
%%bigquery
...

In [ ]:
# Réponse :
# - morceau de la requête n°1 : ...
# - morceau de la requête n°2 : ...
# - morceau de la requête n°3 : ...

# Partie 3 - Machine learning

Nous continuons l’exploration des données avec la création d’un modèle de machine learning, dans BigQuery. Ce sera un modèle de clustering, opérant sur les vecteurs.

On aurait aimé réutiliser notre fonction de distance `euclidean2`, mais BigQuery ML n’offre malheureusement pas la possibilité de personnaliser celle-ci. On va donc utiliser la distance euclidienne fournie.

## Entraînement du modèle

Le K-means de BigQuery suppose que les vecteurs sont sous forme tabulaire, et pas de tableaux imbriqués comme on en a manipulé jusqu’à présent. il faut transformer les données avant d’entraîner le modèle (on peut voir ça comme une étape de feature engineering).

Nous devons donc “pivoter” les vecteurs pour en faire des colonnes. Mais il faut d’abord les “pivoter” en ligne ! On fournit la requête de pivotage dans le notebook de prise en main, vous pouvez l’exécuter pour voir son résultat.

Maintenant que l’on sait pivoter les données, entraîner un modèle de type K-means, en utilisant le squelette proposé par le notebook de prise en main, sur tout le jeu de données pivoté. **La transformation doit spécifier que seules les colonnes V0 à V15 sont utilisées**.

Les options du modèle doivent être :
- `MODEL_TYPE = 'KMEANS'`
- `NUM_CLUSTERS = 5`
- `KMEANS_INIT_METHOD = 'KMEANS++'`
- `DISTANCE_TYPE = 'euclidean'`
- `STANDARDIZE_FEATURES = FALSE`

Vous pouvez aller dans l'interface de BigQuery voir le détail du modèle et accéder à des statistiques sur l’entraînement, les centroïdes des clusters, ...

In [ ]:
%%bigquery
...

## Inférence du modèle

Appliquer ensuite la fonction d’inférence sur une requête pivot similaire appliquée à `flat_queries`, et observer le résultat.

Noter que le résultat ne peut pas servir à identifier facilement un morceau, car on obtient la distance aux centroïdes du modèle, qu'on ne sait pas vraiment interpréter...

In [ ]:
%%bigquery
...

## Exploitation du clustering sur la base de référence

### Récupération
Appliquer l'inférence non plus sur `flat_queries`, mais sur `vectors_with_metadata` elle-même (la table qui a servi à l'entraînement). Récupérer le résultat dans un dataframe.

In [ ]:
%%bigquery clustering_vmd
...

Observer la structure du dataframe. Deux informations vont nous intéresser :
- `song_id`, `window_id`, `artist`, etc. : informations de la fenêtre provenant d'un morceau classé
- `CENTROID_ID` : n° du cluster auquel appartient la fenêtre

Dans la suite, on ne fait plus appel à BigQuery mais on manipule le dataframe mémorisé à l'étape précédente, avec Pandas.

Construire un nouveau dataframe avec une ligne par morceau et les colonnes suivantes :
- `song_id` : ID du morceau
- `artist` : nom de l'artiste
- `centroid_id` : n° du cluster le plus fréquent pour ce morceau (associé au plus grand nombre de fenêtres)

Indications : faire un regroupement du dataframe par morceau et artiste, et appliquer au regroupement une fonction qui détermine le "mode" de la série `CENTROID_ID` (doc Pandas ici : https://pandas.pydata.org/docs/reference/api/pandas.Series.mode.html).

In [ ]:
...

Utiliser ensuite ce dataframe pour répondre aux questions suivantes (**merci de donner le code Python en plus des réponses brutes**).

**Question** : Quelles sont les nombres de morceaux par cluster ? Comment interpréter le fait que certains clusters ne soient pas du tout représentés ?

**Question** : Quels sont les artistes du cluster n°1, avec combien de morceaux chacun ?

**Question** : De la même manière qu'on a déterminé le cluster le plus représenté pour chaque morceau, classer les artistes par cluster le plus représenté pour leurs morceaux (en utilisant le dernier dataframe construit donc).

**Question** : Le clustering des artistes n'est pas très convaincant (si vous les connaissez aussi, vous constaterez que les clusters mélangent un peu tout et n'importe quoi). En reconsidérant tout le cas d'usage, proposer des raisons possibles. **Vous pouvez répondre à cette question même si vous n'avez pas fait les précédentes !**